# SR

In [ ]:
!wget https://huggingface.co/Larvik/LDMjit/resolve/main/dm_pnnx.pt
!wget https://huggingface.co/Larvik/LDMjit/resolve/main/fsd_pnnx.pt
!wget https://huggingface.co/Larvik/LDMjit/resolve/main/alphas_cumprod.npy

import os
import sys
import time

import numpy as np
import cv2
import functools
import torch



alphas_cumprod = np.load('alphas_cumprod.npy')


torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True


# ======================
# Arguemnt Parser Config
# ======================

def imread(filename, flags=cv2.IMREAD_COLOR):
    if not os.path.isfile(filename):
        print(f"File does not exist: {filename}")
        sys.exit()
    data = np.fromfile(filename, np.int8)
    img = cv2.imdecode(data, flags)
    return img

def preprocessing_img(img):
    if len(img.shape) < 3:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGRA)
    elif img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    elif img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGRA)
    return img


def load_image(image_path):
    if os.path.isfile(image_path):
        img = imread(image_path, cv2.IMREAD_UNCHANGED)
    else:
        print(f'{image_path} not found.')
    return preprocessing_img(img)



def im2col(images, filters, stride=1, pad=0):
    if images.ndim == 2:
        images = images.reshape(1, 1, *images.shape)
    elif images.ndim == 3:
        B, I_h, I_w = images.shape
        images = images.reshape(B, 1, I_h, I_w)
    B, C, I_h, I_w = images.shape

    if isinstance(filters, tuple):
        if len(filters) == 2:
            filters = (1, 1, *filters)
        elif len(filters) == 3:
            M, F_h, F_w = filters
            filters = (M, 1, F_h, F_w)
        _, _, F_h, F_w = filters
    else:
        if filters.ndim == 2:
            filters = filters.reshape(1, 1, *filters.shape)
        elif filters.ndim == 3:
            M, F_h, F_w = filters.shape
            filters = filters.reshape(M, 1, F_h, F_w)
        _, _, F_h, F_w = filters.shape

    if isinstance(stride, tuple):
        stride_ud, stride_lr = stride
    else:
        stride_ud = stride
        stride_lr = stride
    if isinstance(pad, tuple):
        pad_ud, pad_lr = pad
    elif isinstance(pad, int):
        pad_ud = pad
        pad_lr = pad
    elif pad == "same":
        pad_ud = 0.5 * ((I_h - 1) * stride_ud - I_h + F_h)
        pad_lr = 0.5 * ((I_w - 1) * stride_lr - I_w + F_w)
    pad_zero = (0, 0)

    O_h = int((I_h - F_h + 2 * pad_ud) // stride_ud + 1)
    O_w = int((I_w - F_w + 2 * pad_lr) // stride_lr + 1)

    result_pad = (pad_ud, pad_lr)
    pad_ud = int(np.ceil(pad_ud))
    pad_lr = int(np.ceil(pad_lr))
    pad_ud = (pad_ud, pad_ud)
    pad_lr = (pad_lr, pad_lr)
    images = np.pad(
        images, [pad_zero, pad_zero, pad_ud, pad_lr], "constant")

    cols = np.empty((B, C, F_h, F_w, O_h, O_w))
    for h in range(F_h):
        h_lim = h + stride_ud * O_h
        for w in range(F_w):
            w_lim = w + stride_lr * O_w
            cols[:, :, h, w, :, :] = \
                images[:, :, h:h_lim:stride_ud, w:w_lim:stride_lr]

    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(C * F_h * F_w, B * O_h * O_w)

    return cols, (O_h, O_w), result_pad


def col2im(cols, I_shape, O_shape, stride=1, pad=0):
    def get_f_shape(i, o, s, p):
        return int(i + 2 * p - (o - 1) * s)

    if len(I_shape) == 2:
        B = C = 1
        I_h, I_w = I_shape
    elif len(I_shape) == 3:
        C = 1
        B, I_h, I_w = I_shape
    else:
        B, C, I_h, I_w = I_shape
    O_h, O_w = O_shape

    if isinstance(stride, tuple):
        stride_ud, stride_lr = stride
    else:
        stride_ud = stride
        stride_lr = stride
    if isinstance(pad, tuple):
        pad_ud, pad_lr = pad
    elif isinstance(pad, int):
        pad_ud = pad
        pad_lr = pad

    F_h = get_f_shape(I_h, O_h, stride_ud, pad_ud)
    F_w = get_f_shape(I_w, O_w, stride_lr, pad_lr)
    pad_ud = int(np.ceil(pad_ud))
    pad_lr = int(np.ceil(pad_lr))
    cols = cols.reshape(C, F_h, F_w, B, O_h, O_w).transpose(3, 0, 1, 2, 4, 5)
    images = np.zeros((B, C, I_h + 2 * pad_ud + stride_ud - 1, I_w + 2 * pad_lr + stride_lr - 1))

    for h in range(F_h):
        h_lim = h + stride_ud * O_h
        for w in range(F_w):
            w_lim = w + stride_lr * O_w
            images[:, :, h:h_lim:stride_ud, w:w_lim:stride_lr] += cols[:, :, h, w, :, :]

    return images[:, :, pad_ud: I_h + pad_ud, pad_lr: I_w + pad_lr]

def meshgrid(h, w):
    y = np.arange(0, h).reshape(h, 1, 1).repeat(w, axis=1)
    x = np.arange(0, w).reshape(1, w, 1).repeat(h, axis=0)
    arr = np.concatenate([y, x], axis=-1)

    return arr


def delta_border(h, w):
    """
    :param h: height
    :param w: width
    :return: normalized distance to image border,
     wtith min distance = 0 at border and max dist = 0.5 at image center
    """
    lower_right_corner = np.array([h - 1, w - 1]).reshape(1, 1, 2)
    arr = meshgrid(h, w) / lower_right_corner
    dist_left_up = np.min(arr, axis=-1, keepdims=True)
    dist_right_down = np.min(1 - arr, axis=-1, keepdims=True)

    edge_dist = np.min(np.concatenate([dist_left_up, dist_right_down], axis=-1), axis=-1)

    return edge_dist


def get_weighting(h, w, Ly, Lx):
    clip_min_weight = 0.01
    clip_max_weight = 0.5

    weighting = delta_border(h, w)
    weighting = np.clip(weighting, clip_min_weight, clip_max_weight)
    weighting = weighting.reshape(1, h * w, 1).repeat(Ly * Lx, axis=-1)

    return weighting


def get_fold_unfold(x, kernel_size, stride, uf=1, df=1):
    """
    :param x: img of size (bs, c, h, w)
    :return: n img crops of size (n, bs, c, kernel_size[0], kernel_size[1])
    """
    bs, nc, h, w = x.shape

    # number of crops in image
    Ly = (h - kernel_size[0]) // stride[0] + 1
    Lx = (w - kernel_size[1]) // stride[1] + 1

    unfold = functools.partial(im2col, filters=kernel_size, stride=stride)
    if uf == 1 and df == 1:
        fold = functools.partial(
            col2im,
            stride=stride)

        weighting = get_weighting(kernel_size[0], kernel_size[1], Ly, Lx)
        weighting = weighting.reshape((1, 1, kernel_size[0], kernel_size[1], Ly * Lx))

    elif uf > 1 and df == 1:
        fold = functools.partial(
            col2im,
            stride=(stride[0] * uf, stride[1] * uf))

        weighting = get_weighting(kernel_size[0] * uf, kernel_size[1] * uf, Ly, Lx)
        weighting = weighting.reshape((1, 1, kernel_size[0] * uf, kernel_size[1] * uf, Ly * Lx))

    elif df > 1 and uf == 1:
        fold = functools.partial(
            col2im,
            stride=(stride[0] // df, stride[1] // df))

        weighting = get_weighting(kernel_size[0] // df, kernel_size[1] // df, Ly, Lx)
        weighting = weighting.reshape((1, 1, kernel_size[0] // df, kernel_size[1] // df, Ly * Lx))

    else:
        raise NotImplementedError

    return fold, unfold, weighting

# ======================
# Secondaty Functions
# ======================

def make_ddim_timesteps(num_ddim_timesteps, num_ddpm_timesteps):
    c = num_ddpm_timesteps // num_ddim_timesteps
    ddim_timesteps = np.asarray(list(range(0, num_ddpm_timesteps, c)))

    # add one to get the final alpha values right (the ones from first scale to data during sampling)
    steps_out = ddim_timesteps + 1

    return steps_out


def make_ddim_sampling_parameters(alphacums, ddim_timesteps, eta):
    # select alphas for computing the variance schedule
    alphas = alphacums[ddim_timesteps]
    alphas_prev = np.asarray([alphacums[0]] + alphacums[ddim_timesteps[:-1]].tolist())

    # according the the formula provided in https://arxiv.org/abs/2010.02502
    sigmas = eta * np.sqrt((1 - alphas_prev) / (1 - alphas) * (1 - alphas / alphas_prev))

    return sigmas, alphas, alphas_prev


# ======================
# Main functions
# ======================




def normalize_image(image, normalize_type='255'):
    """
    Normalize image
    Parameters
    ----------
    image: numpy array
        The image you want to normalize
    normalize_type: string
        Normalize type should be chosen from the type below.
        - '255': simply dividing by 255.0
        - '127.5': output range : -1 and 1
        - 'ImageNet': normalize by mean and std of ImageNet
        - 'None': no normalization
    Returns
    -------
    normalized_image: numpy array
    """
    if normalize_type == 'None':
        return image
    elif normalize_type == '255':
        return image / 255.0
    elif normalize_type == '127.5':
        return image / 127.5 - 1.0
    elif normalize_type == 'ImageNet':
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image / 255.0
        for i in range(3):
            image[:, :, i] = (image[:, :, i] - mean[i]) / std[i]
        return image
    else:
        pass



def preprocess(img):
    im_h, im_w, _ = img.shape

    up_f = 4
    oh, ow = up_f * im_h, up_f * im_w

    img = normalize_image(img, normalize_type='255')

    c = img * 2 - 1
    c = c.transpose(2, 0, 1)  # HWC -> CHW
    c = np.expand_dims(c, axis=0)
    c = c.astype(np.float32)

    c_up = cv2.resize(img, (ow, oh), interpolation=cv2.INTER_LINEAR)
    c_up = c_up.transpose(2, 0, 1)  # HWC -> CHW
    c_up = np.expand_dims(c_up, axis=0)
    c_up = c_up.astype(np.float32)

    return c_up, c


def postprocess(sample):
    sample = np.clip(sample, -1., 1.)
    sample = (sample + 1.) / 2. * 255
    sample = np.transpose(sample, (1, 2, 0))
    sample = sample[:, :, ::-1]  # RGB -> BGR
    sample = sample.astype(np.uint8)

    return sample


def ddim_sampling(
        models, cond):
    shape = cond.shape
    img = np.random.randn(shape[0] * shape[1] * shape[2] * shape[3]).reshape(shape)
    img = img.astype(np.float32)

    timesteps = ddim_timesteps
    time_range = np.flip(timesteps)
    total_steps = timesteps.shape[0]

    print(f"Running DDIM Sampling with {total_steps} timesteps")

    try:
        from tqdm import tqdm
        iterator = tqdm(time_range, desc='DDIM Sampler', total=total_steps)
    except ModuleNotFoundError:
        def iter_func(a):
            for i, x in enumerate(a):
                print("DDIM Sampler: %s/%s" % (i + 1, len(a)))
                yield x

        iterator = iter_func(time_range)

    for i, step in enumerate(iterator):
        index = total_steps - i - 1
        ts = np.full((shape[0],), step, dtype=np.int64)

        img, pred_x0 = p_sample_ddim(
            models,
            img, cond, ts,
            index=index,
        )
        img = img.astype(np.float32)

    return img


# ddim
def p_sample_ddim(
        models, x, c, t, index,
        temperature=1):
    e_t = apply_model(models, x, t, c)

    alphas = ddim_alphas
    alphas_prev = ddim_alphas_prev
    sqrt_one_minus_alphas = ddim_sqrt_one_minus_alphas
    sigmas = ddim_sigmas

    # select parameters corresponding to the currently considered timestep
    b, *_ = x.shape
    a_t = np.full((b, 1, 1, 1), alphas[index])
    a_prev = np.full((b, 1, 1, 1), alphas_prev[index])
    sigma_t = np.full((b, 1, 1, 1), sigmas[index])
    sqrt_one_minus_at = np.full((b, 1, 1, 1), sqrt_one_minus_alphas[index])

    # current prediction for x_0
    pred_x0 = (x - sqrt_one_minus_at * e_t) / np.sqrt(a_t)

    # direction pointing to x_t
    dir_xt = np.sqrt(1. - a_prev - sigma_t ** 2) * e_t

    noise = sigma_t * np.random.randn(x.size).reshape(x.shape) * temperature
    x_prev = np.sqrt(a_prev) * pred_x0 + dir_xt + noise

    return x_prev, pred_x0


def decode_first_stage(models, z):
    ks = (128, 128)
    stride = (64, 64)
    uf = 4

    bs, nc, h, w = z.shape

    fold, unfold, weighting = get_fold_unfold(z, ks, stride, uf=uf)

    z, o_shape, _ = unfold(z)  # (bn, nc * prod(**ks), L)

    # Reshape to img shape
    z = z.reshape((bs, -1, ks[0], ks[1], z.shape[-1]))  # (bn, nc, ks[0], ks[1], L )
    z = z.astype(np.float32)

    print('first_stage_decode...')

    first_stage_decode = models['first_stage_decode']
    outputs = []
    for i in range(z.shape[-1]):
        x = z[:, :, :, :, i]
        output = first_stage_decode(torch.tensor(x).cuda())
        outputs.append(output[0].cpu().numpy())

    o = np.stack(outputs, axis=-1)  # # (bn, nc, ks[0], ks[1], L)
    o = o * weighting

    # Reverse reshape to img shape
    o = o.reshape((o.shape[0], -1, o.shape[-1]))  # (bn, nc * ks[0] * ks[1], L)
    # stitch crops together
    decoded = fold(o, I_shape=(1, 3, h * uf, w * uf), O_shape=o_shape)

    normalization = fold(weighting, I_shape=(1, 1, h * uf, w * uf), O_shape=o_shape)
    decoded = decoded / normalization  # norm is shape (1, 1, h, w)

    return decoded


# ddpm
def apply_model(models, x_noisy, t, cond):
    ks = (128, 128)
    stride = (64, 64)

    bs, nc, h, w = x_noisy.shape

    fold, unfold, weighting = get_fold_unfold(x_noisy, ks, stride)

    z, o_shape, _ = unfold(x_noisy)  # (bn, nc * prod(**ks), L)
    # Reshape to img shape
    z = z.reshape((bs, -1, ks[0], ks[1], z.shape[-1]))  # (bn, nc, ks[0], ks[1], L )
    z_list = [z[:, :, :, :, i] for i in range(z.shape[-1])]

    c, *_ = unfold(cond)
    c = c.reshape((bs, -1, ks[0], ks[1], c.shape[-1]))  # (bn, nc, ks[0], ks[1], L )
    cond_list = [c[:, :, :, :, i] for i in range(c.shape[-1])]

    # apply model by loop over crops
    diffusion_model = models["diffusion_model"]
    outputs = []
    for i in range(z.shape[-1]):
        x = z_list[i]
        cond = cond_list[i]
        xc = np.concatenate([x, cond], axis=1)
        xc = xc.astype(np.float32)
        
        output = diffusion_model(torch.tensor(xc).cuda(), torch.tensor(t).cuda())
            
       
        outputs.append(output[0].cpu().numpy())

    o = np.stack(outputs, axis=-1)
    o = o * weighting

    # Reverse reshape to img shape
    o = o.reshape((o.shape[0], -1, o.shape[-1]))  # (bn, nc * ks[0] * ks[1], L)
    # stitch crops together
    normalization = fold(weighting, I_shape=(1, 1, h, w), O_shape=o_shape)
    x_recon = fold(o, I_shape=(1, 3, h, w), O_shape=o_shape) / normalization

    return x_recon


def predict(models, img):
    img = img[:, :, ::-1]  # BGR -> RGB

    _, c = preprocess(img)

    samples = ddim_sampling(models, c)

    x_sample = decode_first_stage(models, samples)

    img = postprocess(x_sample[0])

    return img

models = dict(
    first_stage_decode=torch.jit.load('/content/fsd_pnnx.pt').eval().cuda(),
    diffusion_model=torch.jit.load('/content/dm_pnnx.pt').eval().cuda(),
)


In [ ]:

"""
ddim_timesteps
"""
ddim_eta = 1.0
ddim_num_steps = 100
ddpm_num_timesteps = 1000
ddim_timesteps = make_ddim_timesteps(ddim_num_steps, ddpm_num_timesteps)

"""
ddim sampling parameters
"""

ddim_sigmas, ddim_alphas, ddim_alphas_prev = \
    make_ddim_sampling_parameters(
        alphacums=alphas_cumprod,
        ddim_timesteps=ddim_timesteps,
        eta=ddim_eta)

ddim_sqrt_one_minus_alphas = np.sqrt(1. - ddim_alphas)

inputz=['/content/sample_data/zkrp.jpg']

 

for image_path in inputz:
    print(image_path)

    # prepare input data
    img = load_image(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    # inference
    print('Start inference...')
    
    img = predict(models, img)

    # plot result
    savepath = '/content/sample_data/out.png'
    print(f'saved at : {savepath}')
    cv2.imwrite(savepath, img)

print('Script finished successfully.')

# txt2img

In [ ]:
!pip install transformers
!wget https://huggingface.co/Larvik/tempsd/resolve/main/alphas_cumprod.npz
!wget https://huggingface.co/Larvik/temp1/resolve/main/transformer_pnnx.pt
!wget https://huggingface.co/Larvik/temp1/resolve/main/diffusion_emb_pnnx.pt
!wget https://huggingface.co/Larvik/temp1/resolve/main/diffusion_mid_pnnx.pt
!wget https://huggingface.co/Larvik/temp1/resolve/main/diffusion_out_pnnx.pt
!wget https://huggingface.co/Larvik/temp1/resolve/main/autoencoder_pnnx.pt
!wget https://huggingface.co/Larvik/temp1/resolve/main/imgencoder_pnnx.pt

In [ ]:
jit=False #@param {type:'boolean'}

import os
import sys
import time
import random
from threading import Thread
import numpy as np
import cv2
from PIL import Image
import PIL
from transformers import CLIPTokenizer

import torch

torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

alphas_cumprod = np.load('alphas_cumprod.npz')['a']

cudev=torch.device('cuda')


def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))


def stochastic_encode(x0, t, use_original_steps=False, noise=None):
    print('came ddim stochastic_encode')
    # fast, but does not allow for exact reconstruction
    # t serves as an index to gather the correct alphas
 
    sqrt_alphas_cumprod = torch.sqrt(torch.tensor(ddim_alphas))
    sqrt_one_minus_alphas_cumprod = torch.tensor(ddim_sqrt_one_minus_alphas)

    if noise is None:
        noise = torch.randn_like(x0)
    return (extract_into_tensor(sqrt_alphas_cumprod, t, x0.shape) * x0 +
            extract_into_tensor(sqrt_one_minus_alphas_cumprod, t, x0.shape) * noise)

# ======================
# Secondaty Functions
# ======================

def make_ddim_timesteps(num_ddim_timesteps, num_ddpm_timesteps):
    c = num_ddpm_timesteps // num_ddim_timesteps
    ddim_timesteps = np.asarray(list(range(0, num_ddpm_timesteps, c)))

    # add one to get the final alpha values right (the ones from first scale to data during sampling)
    steps_out = ddim_timesteps + 1

    return steps_out


def make_ddim_sampling_parameters(alphacums, ddim_timesteps, eta):
    # select alphas for computing the variance schedule
    alphas = alphacums[ddim_timesteps]
    alphas_prev = np.asarray([alphacums[0]] + alphacums[ddim_timesteps[:-1]].tolist())

    # according the the formula provided in https://arxiv.org/abs/2010.02502
    sigmas = eta * np.sqrt((1 - alphas_prev) / (1 - alphas) * (1 - alphas / alphas_prev))

    return sigmas, alphas, alphas_prev


# ======================
# Main functions
# ======================




# encoder
class BERTEmbedder:
    """ Uses a pretrained BERT tokenizer by huggingface. Vocab size: 30522 (?)"""

    def __init__(self, transformer, max_length=77):
        self.tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
        self.max_length = max_length

        self.transformer = transformer
        

    def encode(self, text, nsamp):
        batch_encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, return_length=True,
                            return_overflowing_tokens=False, padding="max_length", return_tensors="pt")

        tokens = batch_encoding["input_ids"]
        tokens = torch.tensor(tokens.numpy()).expand(nsamp,-1)
        

       
        z = self.transformer(tokens)
        

        
        
        
        return z.cuda()

preimg=None

def ddim_sampling(
        models, 
        cond,  t_start,
        unconditional_guidance_scale=1.0,
        unconditional_conditioning=None):
    
    
    if preimg is not None:
      t_start= int(strength * ddim_num_steps)
      img=stochastic_encode(preimg,  torch.tensor([t_start]*n_samples,device='cpu')).numpy()
    else:
      img = np.random.randn(shape[0] * shape[1] * shape[2] * shape[3]).reshape(shape).astype(np.float32)

    timesteps = ddim_timesteps[:t_start]
    time_range = np.flip(timesteps)
    total_steps = timesteps.shape[0]

    print(f"Running DDIM Sampling with {total_steps} timesteps")

    try:
        from tqdm import tqdm
        iterator = tqdm(time_range, desc='DDIM Sampler', total=total_steps)
    except ModuleNotFoundError:
        def iter_func(a):
            for i, x in enumerate(a):
                print("DDIM Sampler: %s/%s" % (i + 1, len(a)))
                yield x

        iterator = iter_func(time_range)

    for i, step in enumerate(iterator):
        index = total_steps - i - 1
        ts = np.full((shape[0],), step, dtype=np.int64)

        img, pred_x0 = p_sample_ddim(
            models,
            img, cond, ts,
            index=index,
            unconditional_guidance_scale=unconditional_guidance_scale,
            unconditional_conditioning=unconditional_conditioning,
        )
        img = img.astype(np.float32)

    return img


# ddim
def p_sample_ddim(
        models, x, c, t, index,
        temperature=1.,
        unconditional_guidance_scale=1.,
        unconditional_conditioning=None):
    x_in = torch.tensor(np.concatenate([x] * 2) ,device=cudev)
    t_in = torch.tensor(np.concatenate([t] * 2) ,device=cudev)
    c_in = torch.cat( ( unconditional_conditioning , c) )

    x_recon = apply_model(models, x_in, t_in, c_in).cpu().numpy()
    e_t_uncond, e_t = np.split(x_recon, 2)

    e_t = e_t_uncond + unconditional_guidance_scale * (e_t - e_t_uncond)

    alphas = ddim_alphas
    alphas_prev = ddim_alphas_prev
    sqrt_one_minus_alphas = ddim_sqrt_one_minus_alphas
    sigmas = ddim_sigmas

    # select parameters corresponding to the currently considered timestep
    b, *_ = x.shape
    a_t = np.full((b, 1, 1, 1), alphas[index])
    a_prev = np.full((b, 1, 1, 1), alphas_prev[index])
    sigma_t = np.full((b, 1, 1, 1), sigmas[index])
    sqrt_one_minus_at = np.full((b, 1, 1, 1), sqrt_one_minus_alphas[index])

    # current prediction for x_0
    pred_x0 = (x - sqrt_one_minus_at * e_t) / np.sqrt(a_t)

    # direction pointing to x_t
    dir_xt = np.sqrt(1. - a_prev - sigma_t ** 2) * e_t

    noise = sigma_t * np.random.randn(x.size).reshape(x.shape) * temperature
    x_prev = np.sqrt(a_prev) * pred_x0 + dir_xt + noise

    return x_prev, pred_x0


# ddpm
def apply_model(models, x, t, cc):
    diffusion_emb = models["diffusion_emb"]
    diffusion_mid = models["diffusion_mid"]
    diffusion_out = models["diffusion_out"]

    with torch.cuda.amp.autocast(dtype=torch.float16):
      output = diffusion_emb(x, t, cc)
      
      h, emb, hs = output
      '''
      print(h.shape)
      print(emb.shape)
      
      for aaa in hs:
        print(aaa.shape)
      '''
      h = diffusion_mid(h, emb, cc, *hs[6:])

      #print(h.shape)

      output = diffusion_out(h, emb, cc, *hs[:6])
    #print(output.shape)

    return output


# decoder
def decode_first_stage(models, z):
    scale_factor = 0.18215
    z = z / scale_factor

    autoencoder = models['autoencoder']
    
    
    output = autoencoder(z)
   
    
    return output

def saver():
  global x_samples_ddim
  np.save( (outputp%(iita,1))[:-6] + '.npy', samples)
  x_samples_ddim = np.clip((x_samples_ddim.numpy() + 1.0) / 2.0, a_min=0.0, a_max=1.0)
  k=0
  for x_sample in x_samples_ddim:
      x_sample = x_sample.transpose(1, 2, 0)  # CHW -> HWC
      x_sample = x_sample * 255
      img = x_sample.astype(np.uint8)
      img = img[:, :, ::-1]  # RGB -> BGR
      cv2.imwrite(outputp%(iita,k), img)
      k+=1
  

    
def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w2, h2 = map(lambda x: x - x % 32, (w, h))
    if w!=w2 or h!=h2:
      image = image.resize((w2, h2), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

def predict(
        models, cond_stage_model,
        prompt, uc):
    global x_samples_ddim
    global samples
    

    c = cond_stage_model.encode(prompt, n_samples)
    
    
   
    samples = ddim_sampling(
        models, c, ddim_num_steps,
        unconditional_guidance_scale=scale,
        unconditional_conditioning=uc)

    
    x_samples_ddim = decode_first_stage(models, torch.tensor(samples, device=cudev) ).cpu()
    t3 = Thread(target = saver)
    a3 = t3.start()

    return


def init_img_type():
  global init_img
  if init_img.endswith('npy'):
    return 0
  elif init_img.endswith('jpg') or init_img.endswith('png'):
    if os.path.isfile(init_img+'.npy'):
      init_img+='.npy'
      return 0
    else:
      return 1
  else:
    return 99



models = dict(
    transformer = torch.jit.load('transformer_pnnx.pt').eval(),
    diffusion_emb = torch.jit.load('diffusion_emb_pnnx.pt').eval().cuda().half(),
    diffusion_mid = torch.jit.load('diffusion_mid_pnnx.pt').eval().cuda().half(),
    diffusion_out = torch.jit.load('diffusion_out_pnnx.pt').eval().cuda().half(),
    autoencoder = torch.jit.load('autoencoder_pnnx.pt').eval().cuda(),
)
    
imgenc=torch.jit.load('imgencoder_pnnx.pt').eval()


In [ ]:
import PIL
init_img='xxx' #@param {type:'string'}
initymgtyp=init_img_type()
if initymgtyp == 0:
  preimg=torch.tensor(np.load(init_img), device='cpu')
  n_samples=preimg.size(0)
  H=preimg.size(2)<<3
  W=preimg.size(3)<<3
elif initymgtyp == 1:
  n_samples=1
  rpt=load_img(init_img)
  H=rpt.size(2)
  W=rpt.size(3)
  preimg=imgenc(  rpt, torch.randn(torch.Size([n_samples,4,H>>3,W>>3]))  )*0.18215
  np.save(init_img+'.npy',preimg.numpy())
else:
  preimg=None

In [ ]:
InThread=False #@param {type:'boolean'}

prompt = 'a photograph of an astronaut riding a horse' #@param {type:'string'}


if preimg is None:
  n_samples = 1 #@param {type:'integer'}
  H=768 #@param {type:'integer'}
  W=704 #@param {type:'integer'}



ddim_num_steps = 50  #@param {type:'integer'}
ddpm_num_timesteps = 1000

outputp='/content/sample_data' #@param {type:'string'}

n_iter = 1 #@param {type:'integer'}

strength=0.5 #@param {type:'number'}

seed=0 #@param {type:'integer'}

scale = 7.5 #@param {type:'number'}
ddim_eta = 0  #@param {type:'integer'}


outputp=outputp+'/'+str(len(os.listdir(outputp)))+'_%dx%d.png'
"""
ddim_timesteps
"""

ddim_timesteps = make_ddim_timesteps(
    ddim_num_steps, ddpm_num_timesteps)

"""
ddim sampling parameters
"""

ddim_sigmas, ddim_alphas, ddim_alphas_prev = \
    make_ddim_sampling_parameters(
        alphacums=alphas_cumprod,
        ddim_timesteps=ddim_timesteps,
        eta=ddim_eta)

ddim_sqrt_one_minus_alphas = np.sqrt(1. - ddim_alphas)

shape = [n_samples, 4, H>>3 , W>>3 ]


if seed == 0:
  np.random.seed(random.randint(0, 2**32))
else:
  np.random.seed(seed)

cond_stage_model = BERTEmbedder(models['transformer'])


print("prompt: %s" % prompt)



print('Start inference...')
uc = None
if scale != 1.0:
  uc = cond_stage_model.encode([''], n_samples)


  
def wpa():
  global x_samples
  global samples
  global iita
  torch.set_grad_enabled(False)
  

  all_samples = []
  for iita in range(n_iter):
      print("iteration: %s" % (iita + 1))

      
      predict(models, cond_stage_model, prompt, uc)
      
  print('Script finished successfully.')
  torch.cuda.empty_cache()


if InThread:
  t1 = Thread(target = wpa)
  a1 = t1.start()
else:
  wpa()
